In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/cours/iasd-deep-golois/project/
!pip install pybind11
!chmod 777 ./compile.sh
!./compile.sh


In [ ]:
!cp golois.cpython-36m-x86_64-linux-gnu.so golois.so

# Legrand & Rausell, 2éme tournoi

## Best solution inspired from "Mobile Networks for Computer Go" 

In [1]:
!pwd

/tf/project


In [1]:
import os
import math
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
from tensorflow.keras import layers 
from tensorflow.keras import regularizers

from tensorflow.keras.layers import Input, Conv2D, DepthwiseConv2D, Dense, Concatenate, Add, ReLU, BatchNormalization, AvgPool2D, MaxPool2D, GlobalAveragePooling2D, Reshape, Permute, Lambda, Flatten, Activation

import golois

planes = 21
moves = 361
N = 10000
epochs = 500
# epochs = 20
batch=64 
filters = 64
trunk = filters

input_data = np.random.randint(2, size=(N, 19, 19, planes))
input_data = input_data.astype ('float32')

print(input_data.ndim)
print(input_data.shape)

policy = np.random.randint(moves, size=(N,))
policy = keras.utils.to_categorical (policy)

value = np.random.randint(2, size=(N,))
value = value.astype ('float32')

end = np.random.randint(2, size=(N, 19, 19, 2))
end = end.astype ('float32')

groups = np.zeros((N, 19, 19, 1))
groups = groups.astype ('float32')

print ("getValidation", flush = True)
golois.getValidation (input_data, policy, value, end)

def bottleneck_block_mobilenet(x, expand=512, squeeze=128):
# def bottleneck_block_mobilenet(x, expand=96, squeeze=16):
    m = layers.Conv2D(expand, (1,1), kernel_regularizer=regularizers.l2(0.0001), use_bias = False)(x)
    m = layers.BatchNormalization()(m)
    m = layers.Activation('relu')(m)
    m = layers.DepthwiseConv2D((3,3), padding='same', kernel_regularizer=regularizers.l2(0.0001), use_bias = False)(m)
    m = layers.BatchNormalization()(m)
    m = layers.Activation('relu')(m)
    m = layers.Conv2D(squeeze, (1,1), kernel_regularizer=regularizers.l2(0.0001), use_bias = False)(m)
    m = layers.BatchNormalization()(m)
    return layers.Add()([m, x])

def bottleneck_block_shufflenet(tensor, expand=512, squeeze=128):
# def bottleneck_block_shufflenet(tensor, expand, squeeze=64):
    x = gconv(tensor, channels=expand, groups=4)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = channel_shuffle(x, groups=4)
    x = DepthwiseConv2D(kernel_size=3, padding='same', kernel_regularizer=regularizers.l2(0.0001), use_bias = False)(x)
    x = BatchNormalization()(x)

    x = gconv(x, channels=squeeze, groups=4)
    x = BatchNormalization()(x)

    x = Add()([tensor, x])
    output = ReLU()(x)
    return output

def gconv(tensor, channels, groups):
    input_ch = tensor.get_shape().as_list()[-1]
    group_ch = input_ch // groups
    output_ch = channels // groups
    groups_list = []

    for i in range(groups):
        group_tensor = tensor[:, :, :, i * group_ch: (i+1) * group_ch]
        group_tensor = Conv2D(output_ch, 1)(group_tensor)
        groups_list.append(group_tensor)

    output = Concatenate()(groups_list)
    return output

def gconv(tensor, channels, groups):
    input_ch = tensor.get_shape().as_list()[-1]
    group_ch = input_ch // groups
    output_ch = channels // groups
    groups_list = []

    for i in range(groups):
        group_tensor = tensor[:, :, :, i * group_ch: (i+1) * group_ch]
        group_tensor = Conv2D(output_ch, 1)(group_tensor)
        groups_list.append(group_tensor)

    output = Concatenate()(groups_list)
    return output

def channel_shuffle(x, groups):  
    _, width, height, channels = x.get_shape().as_list()
    group_ch = channels // groups

    x = Reshape([width, height, group_ch, groups])(x)
    x = Permute([1, 2, 4, 3])(x)
    x = Reshape([width, height, channels])(x)
    return x

4
(10000, 19, 19, 21)
getValidation


In [2]:

# def getModel ():
#    input = keras.Input(shape=(19, 19, planes), name='board')
#    x = Conv2D(trunk, 1, padding='same', kernel_regularizer=regularizers.l2(0.0001))(input)
#    x = BatchNormalization()(x)
#    x = ReLU()(x)
#    for i in range (blocks):
#        x = bottleneck_block_shufflenet (x, filters, trunk)
#    policy_head = Conv2D(1, 1, activation='relu', padding='same', use_bias = False, kernel_regularizer=regularizers.l2(0.0001))(x)
#    policy_head = Flatten()(policy_head)
#    policy_head = Activation('softmax', name='policy')(policy_head)
#    value_head = GlobalAveragePooling2D()(x)
#    value_head = Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.0001))(value_head)
#    value_head = Dense(1, activation='sigmoid', name='value', kernel_regularizer=regularizers.l2(0.0001))(value_head)
        
#    model = keras.Model(inputs=input, outputs=[policy_head, value_head])
#    return model


input = keras.Input(shape=(19, 19, planes), name='board')

# This one is the mobile net (Thomas' version)
# x = layers.Conv2D(trunk, 1, activation='relu', padding='same')(input)
# for i in range (blocks):
#    x = bottleneck_block_mobilenet(x, filters, trunk)
# This one is the mobile net (Antonio's version)
x = layers.Conv2D(trunk, 1, padding='same',kernel_regularizer=regularizers.l2(0.0001))(input)
x_bifurcation = BatchNormalization()(x)

blocks = 33 # 33
filters = 198 # 200
x = ReLU()(x_bifurcation)
for i in range (blocks):
    x = bottleneck_block_mobilenet(x, filters, trunk)

# In series
# for i in range (blocks):
#    x = bottleneck_block_shufflenet(x, filters, trunk)


# In parallel
# y = ReLU()(x_bifurcation)
# for i in range (blocks):
#    y = bottleneck_block_shufflenet(y, filters, trunk)
# concatenate
# x = layers.concatenate([x, y], axis=-1)

    
# fully convolutional, no dense layer
policy_head = layers.Conv2D(1, 1, activation='relu', padding='same', use_bias = False, kernel_regularizer=regularizers.l2(0.0001))(x)
policy_head = layers.Flatten()(policy_head)
policy_head = layers.Activation('softmax', name='policy')(policy_head)

value_head = layers.GlobalAveragePooling2D()(x)
value_head = layers.Dense(50, activation='relu',kernel_regularizer=regularizers.l2(0.0001))(value_head)
value_head = layers.Dense(1, activation='sigmoid', name='value',kernel_regularizer=regularizers.l2(0.0001))(value_head)

model = keras.Model(inputs=input, outputs=[policy_head, value_head])

model.summary ()

# 2nd Train momentum 0.8; 2nd Train momentum 0.9
# model.compile(optimizer=keras.optimizers.SGD(lr=0.000001, momentum=0.9,nesterov=False),
# 2nd Train
model.compile(optimizer=keras.optimizers.SGD(lr=0.005, momentum=0.9),
              loss={'policy': 'categorical_crossentropy', 'value': 'binary_crossentropy'},
              # 2nd Train
              loss_weights={'policy' : 2.0, 'value' : 1.0},
              # loss_weights={'policy' : 1.0, 'value' : 1.0},
              metrics={'policy': 'categorical_accuracy', 'value': 'mse'})


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
board (InputLayer)              [(None, 19, 19, 21)] 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 19, 19, 64)   1408        board[0][0]                      
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 19, 19, 64)   256         conv2d[0][0]                     
__________________________________________________________________________________________________
re_lu (ReLU)                    (None, 19, 19, 64)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [ ]:
#!cp ./test-12052021_0829_last.h5 ./test-12052021_0829_previous.h5

# 2nd and 3rd Train
model.load_weights("./test-14052021_last.h5")

# learning rate schedule
# def step_decay(epoch):
# 	initial_lrate = 0.000005 # 0.005
# 	drop = 0.1
# 	epochs_drop = 2 # 10.0
# 	lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
# 	return lrate

# def lr_scheduler(epoch):
#  if epoch < 100:
#    return 0.00005
#  elif epoch < 150:
#    return 0.000025
#  elif epoch >= 150:
#    return 0.00001

# checkpoint_path = "training_1/cp.ckpt"
# checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
# cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
#                                                 save_weights_only=True,
#                                                 verbose=1)

# adapt LR as epochs grows
#lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)

# 1st model obtained using : 
# lr_callback = tf.keras.callbacks.LearningRateScheduler(step_decay)


# checkpoint_path = '/checkpoint'
checkpoint_path = "checkpoint/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)


callbacks_list = [
    tf.keras.callbacks.EarlyStopping(
        monitor='policy_categorical_accuracy', patience = 10, verbose = 1, mode = "auto",
        restore_best_weights = True
    ),
    tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_path,
        save_weights_only=True,
        monitor='loss',
        mode='min',
        save_best_only=True,
        verbose =1
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='loss',
        # 1st Train
        # factor = 0.2,
        # patience = 10
        # 2nd Train
        # factor = 0.2,
        # patience =  10       
        # 3rdd Train
        factor = 0.1,
        patience = 3
    ),
    keras.callbacks.TensorBoard(
        log_dir='/tf/my_log_dir',
        histogram_freq=5,
        embeddings_freq=0
    )
]

for i in range (1, epochs + 1):
    print ('epoch ' + str (i))
    golois.getBatch (input_data, policy, value, end, groups)
    history = model.fit(input_data,
                        {'policy': policy, 'value': value}, 
                        epochs=1, 
                        callbacks=callbacks_list,
                        batch_size=batch)
    if (i % 10 == 0):
        golois.getValidation (input_data, policy, value, end)
        val = model.evaluate (input_data,
                              [policy, value], verbose = 0, batch_size=batch)
        print ("val =", val)
    
    if (i %100 == 0):
            model.save ('/tf/my_log_dir/test-15052021_tmp.h5')

# 1st Train
# model.save ('test-14052021_last.h5')
# 2nd Train
model.save ('test-15052021_last_3rdTrain.h5')



epoch 1
157/157 [==============================] - 27s 105ms/step - loss: 4.4571 - policy_loss: 1.8323 - value_loss: 0.5680 - policy_categorical_accuracy: 0.4795 - value_mse: 0.1981

Epoch 00001: loss improved from inf to 4.47047, saving model to checkpoint/cp.ckpt
epoch 2
157/157 [==============================] - 16s 102ms/step - loss: 4.4118 - policy_loss: 1.8109 - value_loss: 0.5655 - policy_categorical_accuracy: 0.4811 - value_mse: 0.1985

Epoch 00001: loss improved from 4.47047 to 4.41176, saving model to checkpoint/cp.ckpt
epoch 3
157/157 [==============================] - 18s 114ms/step - loss: 4.4186 - policy_loss: 1.8172 - value_loss: 0.5597 - policy_categorical_accuracy: 0.4801 - value_mse: 0.1960

Epoch 00001: loss did not improve from 4.41176
epoch 4
157/157 [==============================] - 17s 107ms/step - loss: 4.4734 - policy_loss: 1.8456 - value_loss: 0.5577 - policy_categorical_accuracy: 0.4740 - value_mse: 0.1950

Epoch 00001: loss did not improve from 4.41176
epoc

epoch 67
157/157 [==============================] - 16s 104ms/step - loss: 4.3464 - policy_loss: 1.7926 - value_loss: 0.5366 - policy_categorical_accuracy: 0.4819 - value_mse: 0.1866

Epoch 00001: loss did not improve from 4.30271
epoch 68
157/157 [==============================] - 16s 104ms/step - loss: 4.3560 - policy_loss: 1.7964 - value_loss: 0.5387 - policy_categorical_accuracy: 0.4827 - value_mse: 0.1877

Epoch 00001: loss did not improve from 4.30271
epoch 69
157/157 [==============================] - 16s 104ms/step - loss: 4.3246 - policy_loss: 1.7802 - value_loss: 0.5396 - policy_categorical_accuracy: 0.4916 - value_mse: 0.1879

Epoch 00001: loss did not improve from 4.30271
epoch 70
157/157 [==============================] - 16s 104ms/step - loss: 4.3349 - policy_loss: 1.7872 - value_loss: 0.5359 - policy_categorical_accuracy: 0.4881 - value_mse: 0.1869

Epoch 00001: loss did not improve from 4.30271
val = [4.302454948425293, 1.7728782892227173, 0.5321537256240845, 0.48980000

epoch 101
157/157 [==============================] - 17s 106ms/step - loss: 4.3501 - policy_loss: 1.7948 - value_loss: 0.5361 - policy_categorical_accuracy: 0.4813 - value_mse: 0.1868

Epoch 00001: loss did not improve from 4.30271
epoch 102
157/157 [==============================] - 17s 106ms/step - loss: 4.3513 - policy_loss: 1.7965 - value_loss: 0.5337 - policy_categorical_accuracy: 0.4815 - value_mse: 0.1856

Epoch 00001: loss did not improve from 4.30271
epoch 103
157/157 [==============================] - 17s 105ms/step - loss: 4.3744 - policy_loss: 1.8071 - value_loss: 0.5356 - policy_categorical_accuracy: 0.4756 - value_mse: 0.1866

Epoch 00001: loss did not improve from 4.30271
epoch 104
157/157 [==============================] - 16s 103ms/step - loss: 4.3957 - policy_loss: 1.8152 - value_loss: 0.5409 - policy_categorical_accuracy: 0.4839 - value_mse: 0.1882

Epoch 00001: loss did not improve from 4.30271
epoch 105
157/157 [==============================] - 16s 103ms/step - lo